In [1]:
%run "/home/usriniva/uller_modified/discrete_time/Hanna_simu/simulation/impport_packages.ipynb"    #import all necessary packages - numpy, pandas etc
%run "/home/usriniva/uller_modified/discrete_time/Hanna_simu/simulation/simulation_class.ipynb"    #import all necessary packages - numpy, pandas etc
%run "/home/usriniva/uller_modified/discrete_time/Hanna_simu/simulation/create_world.ipynb"   #import all necessary packages - numpy, pandas etc

In [2]:
no_files= 6
dim_objnames_onlygen= {}


In [3]:
rho_cases = 20
m_cases= 21
maxgen= 1000


full_env_values = create_world(rho_cases, m_cases, maxgen)   # create world (input: generations, no. rhos and no. ms) 
print(full_env_values.shape)

clean_data_env = full_env_values[~np.isnan(full_env_values).any(axis=1)]

(420, 1004)


In [9]:
nof_scenarios = np.shape(clean_data_env)[0]
print('nof_scenarios:', nof_scenarios) 

rho_m_alpha_beta = clean_data_env[0:4]
env_states = clean_data_env[4:]


nof_scenarios: 283


In [5]:
popsize = 1000

# Mutation rates: [epi-memory mutation rate, geno mutation rate]
mutrate = [0.01, 0.01]

# Mutation sizes: [epi-memory mutation size, geno mutation size]
mutsize = [1, 1]

epsilon = 0.05 # phenotypic noise

# Number of dimensions (can be 1 or more)
nof_dimensions = 10
                                 
initial_memory = -3


In [6]:
# Example usage:
dims = get_cube_dims(nof_dimensions)

N = create_cube(popsize, dims=dims, nof_dimensions=nof_dimensions, epsilon=epsilon, initial_memory=initial_memory)

print(N.shape)


(1000, 32)


In [10]:


meanw = np.zeros(maxgen)
meanmemory_g = np.zeros(maxgen)
meanmemory_p = np.zeros(maxgen)
meanneutral_g = np.zeros(maxgen)
meanneutral_p = np.zeros(maxgen)

In [ ]:


# --- Main simulation loop ---
for t in range(0, (maxgen - 1)):

    # Uncomment to trace progress
    # if t % 10 == 0:
    #     print(f"Generation {t}")

    # Environmental adjustment
    for d in range(0, nof_dimensions):
        N[:, dims['dev'][d]] = N[:, dims['pheno'][d]] - env_states[t]

    # Fitness calculation
    dev_combined = np.sqrt(np.sum(N[:, dims['dev']] ** 2, axis=1))
    W = np.exp((-dev_combined ** 2) / 2)
    meanw[t] = np.mean(W)

    # Memory and neutral traits
    meanmemory_g[t] = np.mean(N[:, dims['memory']])
    meanmemory_p[t] = np.mean(1 / (1 + np.exp(-N[:, dims['memory']])))
    meanneutral_g[t] = np.mean(N[:, dims['neutral']])
    meanneutral_p[t] = np.mean(1 / (1 + np.exp(-N[:, dims['neutral']])))

    # --- Offspring generation ---
    offspring = np.zeros((popsize, dims['total_layers']))

    # Sample parents by fitness
    parents_idx = np.random.choice(popsize, size=popsize, p=(W / np.sum(W)))
    offspring[:, :] = N[parents_idx, :]

    # --- Mutations ---
    mutate_memories = np.random.uniform(low=0, high=1, size=popsize) < mutrate[0]
    mutate_geno = np.random.uniform(low=0, high=1, size=(popsize, nof_dimensions)) < mutrate[1]

    # Mutate memory (epigenetic)
    offspring[:, dims['memory']] = (
        offspring[:, dims['memory']]
        + mutate_memories * np.random.normal(0, mutsize[0], popsize)
    )

    # Mutate neutral trait
    offspring[:, dims['neutral']] = (
        offspring[:, dims['neutral']]
        + mutate_memories * np.random.normal(0, mutsize[0], popsize)
    )

    # Mutate genotype
    offspring[:, dims['geno']] = (
        offspring[:, dims['geno']]
        + mutate_geno * np.random.normal(0, mutsize[1], size=(popsize, nof_dimensions))
    )

    if only_genetics:
        print(f"Gen {t}: Running ONLY genetics mode")
    for i in range(0, nof_dimensions):
        offspring[:, dims['pheno'][i]] = (
            offspring[:, dims['geno'][i]]
            + np.random.normal(0, epsilon, size=popsize)
        )
    else:
        print(f"Gen {t}: Running EPIGENETIC mode")
        for i in range(0, nof_dimensions):
            offspring[:, dims['pheno'][i]] = (
                offspring[:, dims['geno'][i]]
                + np.random.normal(0, epsilon, size=popsize)
                + ((1 / (1 + np.exp(-offspring[:, dims['memory']])))
                * (N[parents_idx, dims['pheno'][i]] - N[parents_idx, dims['geno'][i]]))
            )

    # Update population
    N = offspring

# --- Collect outputs for inspection ---
sim_result = {
    'maxgen_popsize': [maxgen, popsize],
    'rho_m_alpha_beta': rho_m_alpha_beta,
    'meanw': meanw,
    'meanmemory_g': meanmemory_g,
    'meanmemory_p': meanmemory_p,
    'meanneutral_g': meanneutral_g,
    'meanneutral_p': meanneutral_p
}

# Optional: quick debug checks
print("Simulation finished.")
print("Mean final fitness:", meanw[-1])
print("Mean epigenetic memory (phenotype):", meanmemory_p[-1])


In [15]:
def run_simulation(args):
    env_param_space, popsize, maxgen, dims, N, mutsize, mutrate, epsilon, nof_dimensions, only_genetics = args
    
    rho_m_alpha_beta = env_param_space[0:4]
    env_states = env_param_space[4:]

    meanw = np.zeros(maxgen)
    meanmemory_g = np.zeros(maxgen)
    meanmemory_p= np.zeros(maxgen)
    meanneutral_g = np.zeros(maxgen)
    meanneutral_p = np.zeros(maxgen)
    
    for t in range(0, (maxgen-1)):
        #if t%10 == 0:
        #    print(t)
        for d in range(0, nof_dimensions):
            N[:,dims['dev'][d]] = N[:,dims['pheno'][d]] - env_states[t]
            
        dev_combined = np.sqrt(np.sum(N[:,dims['dev']]**2, axis =1))     
        W = np.exp((-dev_combined**2)/ (2))
        
        
        meanw[t] = np.mean(W)    
        
        meanmemory_g[t] = np.mean(N[:, dims['memory']])  # the actual memory
        meanmemory_p[t] = np.mean(1/(1 + np.exp(-N[:,dims['memory']]))) #with the logistc correction
        
        meanneutral_g[t] = np.mean(N[:,dims['neutral']])
        meanneutral_p[t] = np.mean(1/(1 + np.exp(-N[:,dims['neutral']])))
        
        #current gen offspring (the cube gets stored here per gen)
        
        offspring = np.zeros((popsize, dims['total_layers'])) # empty 3D matrix with dimensions of the cube
        
        #sample offspring for each scenario weighted by fitness
        
        
        parents_idx = np.random.choice(popsize, size=popsize, p = (W/np.sum(W)) )#pick #popsize sized random numbers
        offspring[:,:] = N[parents_idx, :] #similar to how it works in matlab
        
        mutate_memories = np.random.uniform(low=0, high=1, size= popsize) < mutrate[0]
        #mutate_neutral = np.random.uniform(low=0, high=1, size=popsize) < mutrate[0]
        mutate_geno = np.random.uniform(low=0, high=1, size=(popsize, nof_dimensions)) < mutrate[1]
        
        # adding a mutation of size mutsize[1], rate mutate_memories, to the epi-memory
        # the epignetic memory evolves; 
        
        offspring[:,dims['memory']] = (
                                offspring[:,dims['memory']] 
                                + mutate_memories * np.random.normal(0, mutsize[0], popsize)
        )
        
        # adding a mutation of size mutsize[1] (epi-mutation size), rate mutate_memories, to the neutral phenotype
        # the neutral trait recieves random mutation but is not implemented in the calculation for fitness; so is evolving neutrally
        offspring[:,dims['neutral']] = (
                                offspring[:,dims['neutral']]  
                                + mutate_memories * np.random.normal(0, mutsize[0], popsize)
        ) 
        
        # adding a mutation of size mutsize[2] (genetic mutation size), rate mutate_memories, to the genotype

        offspring[:,dims['geno']] = (
                            offspring[:,dims['geno']] 
                            + mutate_geno * np.random.normal(0, mutsize[1], size=(popsize, nof_dimensions) )
        )
        
        if only_genetics:
            print(f"Gen {t}: Running ONLY genetics mode")
            for i in range(0, nof_dimensions):
                offspring[:, dims['pheno'][i]] = (
                offspring[:, dims['geno'][i]]
                + np.random.normal(0, epsilon, size=popsize) )
        else:
            print(f"Gen {t}: Running EPIGENETIC mode")
            for i in range(0, nof_dimensions):
                    offspring[:, dims['pheno'][i]] = (
                        offspring[:, dims['geno'][i]]
                        + np.random.normal(0, epsilon, size=popsize)
                        + ((1 / (1 + np.exp(-offspring[:, dims['memory']])))
                        * (N[parents_idx, dims['pheno'][i]] - N[parents_idx, dims['geno'][i]]))
                    )
        
        
    return {                ## Make sure this is indented!!
        'maxgen_popsize': [maxgen,popsize],
        'rho_m_alpha_beta' :rho_m_alpha_beta,
        'meanw': meanw,
        'meanmemory_g': meanmemory_g,
        'meanmemory_p': meanmemory_p,
        'meanneutral_g': meanneutral_g,
        'meanneutral_p': meanneutral_p
        }
        

In [17]:
rho_cases = 20
m_cases= 21
maxgen= 10


full_env_values = create_world(rho_cases, m_cases, maxgen)   # create world (input: generations, no. rhos and no. ms) 
print(full_env_values.shape)

clean_data_env = full_env_values[~np.isnan(full_env_values).any(axis=1)]

nof_scenarios = np.shape(clean_data_env)[0]
print('nof_scenarios:', nof_scenarios) 

rho_m_alpha_beta = clean_data_env[0:4]
env_states = clean_data_env[4:]

(420, 14)
nof_scenarios: 283


In [18]:
popsize = 1000

# Mutation rates: [epi-memory mutation rate, geno mutation rate]
mutrate = [0.01, 0.01]

# Mutation sizes: [epi-memory mutation size, geno mutation size]
mutsize = [1, 1]

epsilon = 0.05 # phenotypic noise

# Number of dimensions (can be 1 or more)
nof_dimensions = 10
                                 
initial_memory = -3

In [20]:

if __name__ == '__main__':
    
    # Create all combinations of sigma_mut and sigma_alpha
    param_grid = [
        (scenario, popsize, maxgen, dims, N, mutsize, mutrate, epsilon, nof_dimensions, False) #only genetics can either be True or false
        for _, scenario in enumerate(clean_data_env) #where each row corresponds to one scenario
    ]

    num_cpus = 10
    
    with concurrent.futures.ProcessPoolExecutor(num_cpus) as executor: 
        results_array = list(executor.map(run_simulation, param_grid))


Gen 0: Running EPIGENETIC modeGen 0: Running EPIGENETIC mode

Gen 1: Running EPIGENETIC modeGen 1: Running EPIGENETIC mode

Gen 0: Running EPIGENETIC mode
Gen 0: Running EPIGENETIC modeGen 2: Running EPIGENETIC modeGen 2: Running EPIGENETIC modeGen 0: Running EPIGENETIC modeGen 0: Running EPIGENETIC modeGen 0: Running EPIGENETIC mode




Gen 0: Running EPIGENETIC modeGen 1: Running EPIGENETIC modeGen 0: Running EPIGENETIC modeGen 0: Running EPIGENETIC mode

Gen 1: Running EPIGENETIC modeGen 3: Running EPIGENETIC modeGen 3: Running EPIGENETIC modeGen 1: Running EPIGENETIC mode


Gen 1: Running EPIGENETIC mode



Gen 1: Running EPIGENETIC mode
Gen 1: Running EPIGENETIC modeGen 1: Running EPIGENETIC mode
Gen 1: Running EPIGENETIC modeGen 2: Running EPIGENETIC modeGen 4: Running EPIGENETIC modeGen 4: Running EPIGENETIC mode

Gen 2: Running EPIGENETIC mode
Gen 2: Running EPIGENETIC mode


Gen 2: Running EPIGENETIC mode


Gen 3: Running EPIGENETIC modeGen 2: Running EPIGENETIC modeGen 5: Run

In [ ]:
args1 = (env_param_space, popsize, maxgen, dims, N.copy(), mutsize, mutrate, epsilon, nof_dimensions, True)
args2 = (env_param_space, popsize, maxgen, dims, N.copy(), mutsize, mutrate, epsilon, nof_dimensions, False)

res_genetic = run_simulation(args1)
res_epi = run_simulation(args2)

plt.plot(res_genetic['meanw'], label='Genetic only')
plt.plot(res_epi['meanw'], label='Epigenetic enabled')
plt.legend()
plt.title("Fitness comparison")
plt.show()